# Multi-Layers Perceptron
The goal of this notebook is to showcase how to code a multilayer perceptron in Python from scratch.
It makes use of the Perceptron algorithm we developped in the perceptron jupyter notebook with modification to use backpropagation!

We have two main class:
- Neuron: Is used to model one neuron, most of the computation happens there
- MultiLayerPerceptron: Is used to model the full neural network. Here we only support fully connected neural network.
- In this notebook we will code it in such a way that we can have a variable architecture.

The one limitation here is that we still only have one output that is possible 1 or 0. We still didn't got around to have arbitrary amount of output (getting there).

In [ ]:
import random
import math

class Neuron():
    '''
        A conceptual Neuron hat can be trained using a 
        fit and predict methodology, without any library
    '''
    
    def __init__(self, position_in_layer, is_output_neuron=False):
        self.weights = []
        self.inputs = []
        self.output = None
        
        # This is used for the backpropagation update
        self.updated_weights = []
        # This is used to know how to update the weights
        self.is_output_neuron = is_output_neuron
        # This delta is used for the update at the backpropagation
        self.delta = None
        # This is used for the backpropagation update
        self.position_in_layer = position_in_layer 
        
    def attach_to_output(self, neurons):
        '''
            Helper function to store the reference of the other neurons
            To this particular neuron (used for backpropagation)
        '''
        
        self.output_neurons = neurons
    
    def sigmoid(self, x):
        '''
            simple sigmoid function (logistic) used for the activation
        '''
        return 1 / (1 + math.exp(-x))
    
    def init_weights(self, num_input):
        '''
            This is used to setup the weights when we know how many inputs there is for
            a given neuron
        '''
        
        # Randomly initalize the weights
        for i in range(num_input+1):
            self.weights.append(random.uniform(0,1))
        
    def predict(self, row):
        '''
            Given a row of data it will predict what the output should be for
            this given neuron. We can have many input, but only one output for a neuron
        '''
        
        # Reset the inputs
        self.inputs = []
        
        # We iterate over the weights and the features in the given row
        activation = 0
        for weight, feature in zip(self.weights, row):
            self.inputs.append(feature)
            activation = activation + weight*feature
            
        
        self.output = self.sigmoid(activation)
        return self.output
    
        
            
    def update_neuron(self):
        '''
            Will update a given neuron weights by replacing the current weights
            with those used during the backpropagation. This need to be done at the end of the
            backpropagation
        '''
        
        self.weights = []
        for new_weight in self.updated_weights:
            self.weights.append(new_weight)
    
    def calculate_update(self, learning_rate, target):
        '''
            This function will calculate the updated weights for this neuron. It will first calculate
            the right delta (depending if this neuron is a ouput or a hidden neuron), then it will
            calculate the right updated_weights. It will not overwrite the weights yet as they are needed
            for other update in the backpropagation algorithm.
        '''
        
        if self.is_output_neuron:
            # Calculate the delta for the output
            self.delta = (self.output - target)*self.output*(1-self.output)
        else:
            # Calculate the delta
            delta_sum = 0
            # this is to know which weights this neuron is contributing in the output layer
            cur_weight_index = self.position_in_layer 
            for output_neuron in self.output_neurons:
                delta_sum = delta_sum + (output_neuron.delta * output_neuron.weights[cur_weight_index])

            # Update this neuron delta
            self.delta = delta_sum*self.output*(1-self.output)
            
            
        # Reset the update weights
        self.updated_weights = []
        
        # Iterate over each weight and update them
        for cur_weight, cur_input in zip(self.weights, self.inputs):
            gradient = self.delta*cur_input
            new_weight = cur_weight - learning_rate*gradient
            self.updated_weights.append(new_weight)
         

In [ ]:
class Layer():
    '''
        Layer is modelizing a layer in the fully-connected-feedforward neural network architecture.
        It will play the role of connecting everything together inside and will be doing the backpropagation 
        update.
    '''
    
    def __init__(self, num_neuron, is_output_layer = False):
        
        # Will create that much neurons in this layer
        self.is_output_layer = is_output_layer
        self.neurons = []
        for i in range(num_neuron):
            # Create neuron
            neuron = Neuron(i,  is_output_neuron=is_output_layer)
            self.neurons.append(neuron)
    
    def attach(self, layer):
        '''
            This function attach the neurons from this layer to another one
            This is needed for the backpropagation algorithm
        '''
        # Iterate over the neurons in the current layer and attach 
        # them to the next layer
        for in_neuron in self.neurons:
            in_neuron.attach_to_output(layer.neurons)
            
    def init_layer(self, num_input):
        '''
            This will initialize the weights of each neuron in the layer.
            By giving the right num_input it will spawn the right number of weights
        '''
        
        # Iterate over each of the neuron and initialize
        # the weights that connect with the previous layer
        for neuron in self.neurons:
            neuron.init_weights(num_input)
    
    def predict(self, row):
        '''
            This will calcualte the activations for the full layer given the row of data 
            streaming in.
        '''
        row.append(1) # need to add the bias
        activations = [neuron.predict(row) for neuron in self.neurons]
        return activations
            
        

In [ ]:
class MultiLayerPerceptron():
    '''
        We will be creating the multi-layer perceptron with only two layer:
        an input layer, a perceptrons layer and a one neuron output layer which does binary classification
    '''
    def __init__(self, learning_rate = 0.01, num_iteration = 100):
        
        # Layers
        self.layers = []
                
        # Training parameters
        self.learning_rate = learning_rate
        self.num_iteration = num_iteration
        
        
    def add_output_layer(self, num_neuron):
        '''
            This helper function will create a new output layer and add it to the architecture
        '''
        self.layers.insert(0, Layer(num_neuron, is_output_layer = True))
    
    def add_hidden_layer(self, num_neuron):
        '''
            This helper function will create a new hidden layer, add it to the architecture
            and finally attach it to the front of the architecture
        '''
        # Create an hidden layer
        hidden_layer = Layer(num_neuron)
        # Attach the last added layer to this new layer
        hidden_layer.attach(self.layers[0])
        # Add this layers to the architecture
        self.layers.insert(0, hidden_layer)
        
    def update_layers(self, target):
        '''
            Will update all the layers by calculating the updated weights and then updating 
            the weights all at once when the new weights are found.
        '''
        # Iterate over each of the layer in reverse order
        # to calculate the updated weights
        for layer in reversed(self.layers):
                           
            # Calculate update the hidden layer
            for neuron in layer.neurons:
                neuron.calculate_update(self.learning_rate, target)  
        
        # Iterate over each of the layer in normal order
        # to update the weights
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.update_neuron()
    
    def fit(self, X, y):
        '''
            Main training function of the neural network algorithm. This will make use of backpropagation.
            It will use stochastic gradient descent by selecting one row at random from the dataset and 
            use predict to calculate the error. The error will then be backpropagated and new weights calculated.
            Once all the new weights are calculated, the whole network weights will be updated
        '''
        num_row = len(X)
        num_feature = len(X[0]) # Here we assume that we have a rectangular matrix
        
        # Init the weights throughout each of the layer
        self.layers[0].init_layer(num_feature)
        
        for i in range(1, len(self.layers)):
            num_input = len(self.layers[i-1].neurons)
            self.layers[i].init_layer(num_input)

        # Launch the training algorithm
        for i in range(self.num_iteration):
            
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i] # take the random sample from the dataset
            yhat = self.predict(row)
            target = y[r_i]
            
            # Update the layers using backpropagation   
            self.update_layers(target)
            
            # At every 100 iteration we calculate the error
            # on the whole training set
            if i % 1000 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    
    def predict(self, row):
        '''
            Prediction function that will take a row of input and give back the output
            of the whole neural network.
        '''
        
        # Gather all the activation in the hidden layer
        
        activations = self.layers[0].predict(row)
        for i in range(1, len(self.layers)):
            activations = self.layers[i].predict(activations)

        outputs = []
        for activation in activations:                        
            # Decide if we output a 1 or 0
            if activation >= 0.5:
                outputs.append(1.0)
            else:
                outputs.append(0.0)
                           
        # We currently have only One output allowed
        return outputs[0]

In [5]:
# XOR function (one or the other but not both)
X = [[0,0], [0,1], [1,0], [1,1]]
y = [0, 1, 1, 0]

# Init the parameters for the network
clf = MultiLayerPerceptron(learning_rate = 0.1, num_iteration = 100000)
# Create the architecture backward
clf.add_output_layer(num_neuron = 1)
clf.add_hidden_layer(num_neuron = 3)
clf.add_hidden_layer(num_neuron = 2)
# Train the network
clf.fit(X,y)

3
3
3
3
3
3
3
3
3
3
4
4
3
4
3
4
3
3
3
3
3
3
4
4
3
3
3
3
3
3
3
3
3
3
4
4
3
3
3
3
3
3
3
3
3
3
4
4
3
3
3
3
3
3
3
3
3
3
4
4
Iteration 0 with error = 0.5
3
4
3
4
3
3
3
3
3
3
4
4
3
5
3
5
3
3
3
3
3
3
4
4
3
6
3
6
3
3
3
3
3
3
4
4
3
7
3
7
3
3
3
3
3
3
4
4
3
4
3
4
3
3
3
3
3
3
4
4
3
8
3
8
3
3
3
3
3
3
4
4
3
5
3
5
3
3
3
3
3
3
4
4
3
5
3
5
3
3
3
3
3
3
4
4
3
6
3
6
3
3
3
3
3
3
4
4
3
4
3
4
3
3
3
3
3
3
4
4
3
5
3
5
3
3
3
3
3
3
4
4
3
7
3
7
3
3
3
3
3
3
4
4
3
8
3
8
3
3
3
3
3
3
4
4
3
6
3
6
3
3
3
3
3
3
4
4
3
9
3
9
3
3
3
3
3
3
4
4
3
10
3
10
3
3
3
3
3
3
4
4
3
9
3
9
3
3
3
3
3
3
4
4
3
11
3
11
3
3
3
3
3
3
4
4
3
12
3
12
3
3
3
3
3
3
4
4
3
13
3
13
3
3
3
3
3
3
4
4
3
10
3
10
3
3
3
3
3
3
4
4
3
6
3
6
3
3
3
3
3
3
4
4
3
11
3
11
3
3
3
3
3
3
4
4
3
14
3
14
3
3
3
3
3
3
4
4
3
7
3
7
3
3
3
3
3
3
4
4
3
7
3
7
3
3
3
3
3
3
4
4
3
12
3
12
3
3
3
3
3
3
4
4
3
8
3
8
3
3
3
3
3
3
4
4
3
9
3
9
3
3
3
3
3
3
4
4
3
8
3
8
3
3
3
3
3
3
4
4
3
10
3
10
3
3
3
3
3
3
4
4
3
15
3
15
3
3
3
3
3
3
4
4
3
9
3
9
3
3
3
3
3
3
4
4
3
16
3
16
3
3
3
3
3
3
4
4
3
10
3
10
3
3

4
3
110
3
110
3
3
3
3
3
3
4
4
3
132
3
132
3
3
3
3
3
3
4
4
3
111
3
111
3
3
3
3
3
3
4
4
3
88
3
88
3
3
3
3
3
3
4
4
3
109
3
109
3
3
3
3
3
3
4
4
3
112
3
112
3
3
3
3
3
3
4
4
3
110
3
110
3
3
3
3
3
3
4
4
3
133
3
133
3
3
3
3
3
3
4
4
3
113
3
113
3
3
3
3
3
3
4
4
3
134
3
134
3
3
3
3
3
3
4
4
3
89
3
89
3
3
3
3
3
3
4
4
3
90
3
90
3
3
3
3
3
3
4
4
3
114
3
114
3
3
3
3
3
3
4
4
3
91
3
91
3
3
3
3
3
3
4
4
3
111
3
111
3
3
3
3
3
3
4
4
3
135
3
135
3
3
3
3
3
3
4
4
3
112
3
112
3
3
3
3
3
3
4
4
3
115
3
115
3
3
3
3
3
3
4
4
3
136
3
136
3
3
3
3
3
3
4
4
3
116
3
116
3
3
3
3
3
3
4
4
3
113
3
113
3
3
3
3
3
3
4
4
3
117
3
117
3
3
3
3
3
3
4
4
3
118
3
118
3
3
3
3
3
3
4
4
3
114
3
114
3
3
3
3
3
3
4
4
3
115
3
115
3
3
3
3
3
3
4
4
3
119
3
119
3
3
3
3
3
3
4
4
3
92
3
92
3
3
3
3
3
3
4
4
3
116
3
116
3
3
3
3
3
3
4
4
3
120
3
120
3
3
3
3
3
3
4
4
3
121
3
121
3
3
3
3
3
3
4
4
3
122
3
122
3
3
3
3
3
3
4
4
3
93
3
93
3
3
3
3
3
3
4
4
3
94
3
94
3
3
3
3
3
3
4
4
3
137
3
137
3
3
3
3
3
3
4
4
3
138
3
138
3
3
3
3
3
3
4
4
3
95
3
95
3
3
3
3
3
3
4
4
3
123


3
203
3
203
3
3
3
3
3
3
4
4
3
204
3
204
3
3
3
3
3
3
4
4
3
205
3
205
3
3
3
3
3
3
4
4
3
239
3
239
3
3
3
3
3
3
4
4
3
223
3
223
3
3
3
3
3
3
4
4
3
202
3
202
3
3
3
3
3
3
4
4
3
240
3
240
3
3
3
3
3
3
4
4
3
224
3
224
3
3
3
3
3
3
4
4
3
241
3
241
3
3
3
3
3
3
4
4
3
242
3
242
3
3
3
3
3
3
4
4
3
206
3
206
3
3
3
3
3
3
4
4
3
207
3
207
3
3
3
3
3
3
4
4
3
208
3
208
3
3
3
3
3
3
4
4
3
203
3
203
3
3
3
3
3
3
4
4
3
204
3
204
3
3
3
3
3
3
4
4
3
209
3
209
3
3
3
3
3
3
4
4
3
243
3
243
3
3
3
3
3
3
4
4
3
244
3
244
3
3
3
3
3
3
4
4
3
205
3
205
3
3
3
3
3
3
4
4
3
210
3
210
3
3
3
3
3
3
4
4
3
245
3
245
3
3
3
3
3
3
4
4
3
211
3
211
3
3
3
3
3
3
4
4
3
206
3
206
3
3
3
3
3
3
4
4
3
207
3
207
3
3
3
3
3
3
4
4
3
246
3
246
3
3
3
3
3
3
4
4
3
225
3
225
3
3
3
3
3
3
4
4
3
247
3
247
3
3
3
3
3
3
4
4
3
208
3
208
3
3
3
3
3
3
4
4
3
212
3
212
3
3
3
3
3
3
4
4
3
248
3
248
3
3
3
3
3
3
4
4
3
209
3
209
3
3
3
3
3
3
4
4
3
226
3
226
3
3
3
3
3
3
4
4
3
213
3
213
3
3
3
3
3
3
4
4
3
214
3
214
3
3
3
3
3
3
4
4
3
210
3
210
3
3
3
3
3
3
4
4
3
211
3
211
3
3
3
3


3
3
3
4
4
3
327
3
327
3
3
3
3
3
3
4
4
3
333
3
333
3
3
3
3
3
3
4
4
3
328
3
328
3
3
3
3
3
3
4
4
3
334
3
334
3
3
3
3
3
3
4
4
3
295
3
295
3
3
3
3
3
3
4
4
3
296
3
296
3
3
3
3
3
3
4
4
3
325
3
325
3
3
3
3
3
3
4
4
3
326
3
326
3
3
3
3
3
3
4
4
3
327
3
327
3
3
3
3
3
3
4
4
3
297
3
297
3
3
3
3
3
3
4
4
3
329
3
329
3
3
3
3
3
3
4
4
3
298
3
298
3
3
3
3
3
3
4
4
3
335
3
335
3
3
3
3
3
3
4
4
3
299
3
299
3
3
3
3
3
3
4
4
3
336
3
336
3
3
3
3
3
3
4
4
3
300
3
300
3
3
3
3
3
3
4
4
3
337
3
337
3
3
3
3
3
3
4
4
3
338
3
338
3
3
3
3
3
3
4
4
3
339
3
339
3
3
3
3
3
3
4
4
3
340
3
340
3
3
3
3
3
3
4
4
3
330
3
330
3
3
3
3
3
3
4
4
3
328
3
328
3
3
3
3
3
3
4
4
3
329
3
329
3
3
3
3
3
3
4
4
3
341
3
341
3
3
3
3
3
3
4
4
3
330
3
330
3
3
3
3
3
3
4
4
3
301
3
301
3
3
3
3
3
3
4
4
3
302
3
302
3
3
3
3
3
3
4
4
3
342
3
342
3
3
3
3
3
3
4
4
3
331
3
331
3
3
3
3
3
3
4
4
3
303
3
303
3
3
3
3
3
3
4
4
3
332
3
332
3
3
3
3
3
3
4
4
3
333
3
333
3
3
3
3
3
3
4
4
3
304
3
304
3
3
3
3
3
3
4
4
3
331
3
331
3
3
3
3
3
3
4
4
3
332
3
332
3
3
3
3
3
3
4
4
3
305
3
30

3
3
3
3
3
3
4
4
3
439
3
439
3
3
3
3
3
3
4
4
3
418
3
418
3
3
3
3
3
3
4
4
3
419
3
419
3
3
3
3
3
3
4
4
3
420
3
420
3
3
3
3
3
3
4
4
3
421
3
421
3
3
3
3
3
3
4
4
3
442
3
442
3
3
3
3
3
3
4
4
3
422
3
422
3
3
3
3
3
3
4
4
3
443
3
443
3
3
3
3
3
3
4
4
3
423
3
423
3
3
3
3
3
3
4
4
3
440
3
440
3
3
3
3
3
3
4
4
3
424
3
424
3
3
3
3
3
3
4
4
3
463
3
463
3
3
3
3
3
3
4
4
3
464
3
464
3
3
3
3
3
3
4
4
3
425
3
425
3
3
3
3
3
3
4
4
3
426
3
426
3
3
3
3
3
3
4
4
3
444
3
444
3
3
3
3
3
3
4
4
3
427
3
427
3
3
3
3
3
3
4
4
3
428
3
428
3
3
3
3
3
3
4
4
3
429
3
429
3
3
3
3
3
3
4
4
3
441
3
441
3
3
3
3
3
3
4
4
3
430
3
430
3
3
3
3
3
3
4
4
3
442
3
442
3
3
3
3
3
3
4
4
3
445
3
445
3
3
3
3
3
3
4
4
3
431
3
431
3
3
3
3
3
3
4
4
3
432
3
432
3
3
3
3
3
3
4
4
3
443
3
443
3
3
3
3
3
3
4
4
3
444
3
444
3
3
3
3
3
3
4
4
3
445
3
445
3
3
3
3
3
3
4
4
3
465
3
465
3
3
3
3
3
3
4
4
3
446
3
446
3
3
3
3
3
3
4
4
3
466
3
466
3
3
3
3
3
3
4
4
3
447
3
447
3
3
3
3
3
3
4
4
3
467
3
467
3
3
3
3
3
3
4
4
3
448
3
448
3
3
3
3
3
3
4
4
3
468
3
468
3
3
3
3
3
3
4
4
3
46

3
3
3
4
4
3
538
3
538
3
3
3
3
3
3
4
4
3
539
3
539
3
3
3
3
3
3
4
4
3
573
3
573
3
3
3
3
3
3
4
4
3
574
3
574
3
3
3
3
3
3
4
4
3
575
3
575
3
3
3
3
3
3
4
4
3
576
3
576
3
3
3
3
3
3
4
4
3
564
3
564
3
3
3
3
3
3
4
4
3
577
3
577
3
3
3
3
3
3
4
4
3
565
3
565
3
3
3
3
3
3
4
4
3
578
3
578
3
3
3
3
3
3
4
4
3
579
3
579
3
3
3
3
3
3
4
4
3
545
3
545
3
3
3
3
3
3
4
4
3
546
3
546
3
3
3
3
3
3
4
4
3
566
3
566
3
3
3
3
3
3
4
4
3
567
3
567
3
3
3
3
3
3
4
4
3
580
3
580
3
3
3
3
3
3
4
4
3
568
3
568
3
3
3
3
3
3
4
4
3
581
3
581
3
3
3
3
3
3
4
4
3
540
3
540
3
3
3
3
3
3
4
4
3
582
3
582
3
3
3
3
3
3
4
4
3
547
3
547
3
3
3
3
3
3
4
4
3
541
3
541
3
3
3
3
3
3
4
4
3
583
3
583
3
3
3
3
3
3
4
4
3
584
3
584
3
3
3
3
3
3
4
4
3
548
3
548
3
3
3
3
3
3
4
4
3
585
3
585
3
3
3
3
3
3
4
4
3
549
3
549
3
3
3
3
3
3
4
4
3
586
3
586
3
3
3
3
3
3
4
4
3
542
3
542
3
3
3
3
3
3
4
4
3
587
3
587
3
3
3
3
3
3
4
4
3
588
3
588
3
3
3
3
3
3
4
4
3
543
3
543
3
3
3
3
3
3
4
4
3
589
3
589
3
3
3
3
3
3
4
4
3
550
3
550
3
3
3
3
3
3
4
4
3
544
3
544
3
3
3
3
3
3
4
4
3
590
3
59

3
3
3
3
4
4
3
646
3
646
3
3
3
3
3
3
4
4
3
680
3
680
3
3
3
3
3
3
4
4
3
647
3
647
3
3
3
3
3
3
4
4
3
676
3
676
3
3
3
3
3
3
4
4
3
677
3
677
3
3
3
3
3
3
4
4
3
656
3
656
3
3
3
3
3
3
4
4
3
648
3
648
3
3
3
3
3
3
4
4
3
681
3
681
3
3
3
3
3
3
4
4
3
682
3
682
3
3
3
3
3
3
4
4
3
649
3
649
3
3
3
3
3
3
4
4
3
683
3
683
3
3
3
3
3
3
4
4
3
678
3
678
3
3
3
3
3
3
4
4
3
650
3
650
3
3
3
3
3
3
4
4
3
651
3
651
3
3
3
3
3
3
4
4
3
657
3
657
3
3
3
3
3
3
4
4
3
684
3
684
3
3
3
3
3
3
4
4
3
658
3
658
3
3
3
3
3
3
4
4
3
685
3
685
3
3
3
3
3
3
4
4
3
652
3
652
3
3
3
3
3
3
4
4
3
679
3
679
3
3
3
3
3
3
4
4
3
659
3
659
3
3
3
3
3
3
4
4
3
660
3
660
3
3
3
3
3
3
4
4
3
680
3
680
3
3
3
3
3
3
4
4
3
653
3
653
3
3
3
3
3
3
4
4
3
681
3
681
3
3
3
3
3
3
4
4
3
682
3
682
3
3
3
3
3
3
4
4
3
661
3
661
3
3
3
3
3
3
4
4
3
654
3
654
3
3
3
3
3
3
4
4
3
683
3
683
3
3
3
3
3
3
4
4
3
686
3
686
3
3
3
3
3
3
4
4
3
662
3
662
3
3
3
3
3
3
4
4
3
687
3
687
3
3
3
3
3
3
4
4
3
684
3
684
3
3
3
3
3
3
4
4
3
688
3
688
3
3
3
3
3
3
4
4
3
685
3
685
3
3
3
3
3
3
4
4
3
655
3


4
4
3
761
3
761
3
3
3
3
3
3
4
4
3
762
3
762
3
3
3
3
3
3
4
4
3
782
3
782
3
3
3
3
3
3
4
4
3
763
3
763
3
3
3
3
3
3
4
4
3
764
3
764
3
3
3
3
3
3
4
4
3
787
3
787
3
3
3
3
3
3
4
4
3
783
3
783
3
3
3
3
3
3
4
4
3
788
3
788
3
3
3
3
3
3
4
4
3
784
3
784
3
3
3
3
3
3
4
4
3
789
3
789
3
3
3
3
3
3
4
4
3
736
3
736
3
3
3
3
3
3
4
4
3
737
3
737
3
3
3
3
3
3
4
4
3
785
3
785
3
3
3
3
3
3
4
4
3
738
3
738
3
3
3
3
3
3
4
4
3
765
3
765
3
3
3
3
3
3
4
4
3
786
3
786
3
3
3
3
3
3
4
4
3
790
3
790
3
3
3
3
3
3
4
4
3
791
3
791
3
3
3
3
3
3
4
4
3
739
3
739
3
3
3
3
3
3
4
4
3
766
3
766
3
3
3
3
3
3
4
4
3
792
3
792
3
3
3
3
3
3
4
4
3
787
3
787
3
3
3
3
3
3
4
4
3
740
3
740
3
3
3
3
3
3
4
4
3
793
3
793
3
3
3
3
3
3
4
4
3
741
3
741
3
3
3
3
3
3
4
4
3
742
3
742
3
3
3
3
3
3
4
4
3
767
3
767
3
3
3
3
3
3
4
4
3
768
3
768
3
3
3
3
3
3
4
4
3
788
3
788
3
3
3
3
3
3
4
4
3
769
3
769
3
3
3
3
3
3
4
4
3
794
3
794
3
3
3
3
3
3
4
4
3
743
3
743
3
3
3
3
3
3
4
4
3
744
3
744
3
3
3
3
3
3
4
4
3
745
3
745
3
3
3
3
3
3
4
4
3
746
3
746
3
3
3
3
3
3
4
4
3
747
3
747
3
3


KeyboardInterrupt: 

In [12]:
print("Expected 0.0, got: ",clf.predict([0,0]))
print("Expected 1.0, got: ",clf.predict([0,1]))
print("Expected 1.0, got: ",clf.predict([1,0]))
print("Expected 0.0, got: ",clf.predict([1,1]))

Expected 0.0, got:  0.0
Expected 1.0, got:  1.0
Expected 1.0, got:  1.0
Expected 0.0, got:  0.0
